To Do:
- Have scaper identify and download forms in formats other than pdfs (e.g., doc and docx)
- Add more jurisdictions/states

# Load Libraries

In [15]:
import urllib.request, json 
from urllib.request import urlopen
import re
from bs4 import BeautifulSoup
import pandas as pd
import time
from datetime import date
import os
from os import walk
import os.path
from os import path
import uuid

In [16]:
def reCase(text):
    output = re.sub("(\w|\d)(_|-)(\w|\d)","\\1 \\3",text)
    output = re.sub("([a-z])([A-Z]|\d)","\\1 \\2",output)
    output = re.sub("(\d)([A-Z]|[a-z])","\\1 \\2",output)
    return output

In [17]:
import requests

from urllib.request import Request, urlopen
from urllib.parse import quote

#!pip install PyPDF2
import PyPDF2

from io import StringIO # https://stackoverflow.com/a/18284900
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from pdfminer.pdfpage import PDFPage

def convert(fname, pages=None):
    if not pages:
        pagenums = set()
    else:
        pagenums = set(pages)

    output = StringIO()
    manager = PDFResourceManager()
    converter = TextConverter(manager, output, laparams=LAParams())
    interpreter = PDFPageInterpreter(manager, converter)

    infile = open(fname, 'rb') # python three change
    for page in PDFPage.get_pages(infile, pagenums):
        interpreter.process_page(page)
    infile.close()
    converter.close()
    text = output.getvalue()
    output.close
    return text

#def download_pdf(url,path,filename):
#    filename = path+filename
#    try:
#        f = urlopen(url)
#        with open(filename, "wb") as code:
#            code.write(f.read())
#        return filename
#    except:
#        return "error"
    
def download_pdf(url,path,filename):
    try:
        filename = path+filename

        f = requests.get(url, stream=True, headers={'User-agent': 'Mozilla/5.0'})

        with open(filename, "wb") as code:
            #code.write(f.read())
            code.write(f.content)

        return convert(filename)
    except:
        return "error"

# Download forms and meta data from UT courts

Download forms and place them in the folder defined by `form_path` while creating a dataframe that you save to `csv_path`.


In [18]:
jur = "UT"
source = "www.utcourts.gov"
site = "https://www.utcourts.gov/forms/"
form_path = "../data/raw/www.utcourts.gov/forms/"
csv_path = "../data/raw/www.utcourts.gov/"

try:
    files_df = pd.read_csv(csv_path+"form_data.csv")
except:
    files_df = pd.DataFrame([],columns=["id","jurisdiction","source","title","group","url","filename","downloaded"])
    files_df.to_csv(csv_path+"form_data.csv", index=False, encoding="utf-8")   
    
files_df[:1]

,id,jurisdiction,source,title,group,url,filename,downloaded
0,ed1efd08e3304bdaa887db753509d666,UT,www.utcourts.gov,Community Service Worksheet Third District Juv...,3rd District Juvenile Court: Forms and Pamphlets,https://www.utcourts.gov/courts/juv/juvsites/3...,COMMUNITY%20SERVICE%20WORKSHEET-FRONT%20AND%20...,2021-11-03


In [19]:
with urllib.request.urlopen(urllib.request.Request(site)) as url:
    content = url.read().decode()
soup = BeautifulSoup(content)

In [20]:
today = date.today().strftime("%Y-%m-%d")
body = soup.find('div',id="content")
i = 0
limit = 5
for section in re.split("<h3>",str(body), flags=re.IGNORECASE):
    group = re.match('^[^<]*',section)[0]
    
    #print(group)
    
    section_html = BeautifulSoup(section)
    for li in section_html.find_all('li'):
        title = re.sub('\(.*\s*.*','',li.text)
        
        #print("\t",i,title)
        
        for a in li.find_all('a', href=True):
            if (re.search("^http.*\.pdf",a['href'])):
                url = re.search(".*/([^/]*)\.pdf",a['href'], re.IGNORECASE)
                filename = url.group(1)+".pdf"
                fileurl = url.group(0)
                fileid = uuid.uuid4().hex
                
                #print("\t",filename,"\n\t",fileurl,"\n")
                
                status = ""
                if fileurl not in files_df["url"].values:
                    print("Downloading: "+fileurl)
                    if (i < limit):
                        status = download_pdf(fileurl,form_path,fileid+".pdf")

                    if status == "error":
                        print("#######################")
                        print("         Error!!!")
                        print("#######################")
                        #time.sleep(5)
                    else:
                        if (i < limit):
                            files_df = files_df.append(pd.DataFrame([[fileid,jur,source,title,group,fileurl,filename,today]],columns=["id","jurisdiction","source","title","group","url","filename","downloaded"]), ignore_index=True,sort=False)
                            i += 1
                        else:
                            print("skip")
                        #time.sleep(1)

Downloading: https://www.utcourts.gov/howto/family/adoption/stepchild/docs/10_Decree.pdf
Downloading: https://www.utcourts.gov/howto/family/adoption/stepchild/docs/03_1_Consent_Birth_Mother.pdf
Downloading: https://www.utcourts.gov/howto/family/adoption/stepchild/docs/07_Certificate_Readiness.pdf
Downloading: https://www.utcourts.gov/howto/family/adoption/stepchild/docs/13_Motion_Alternative_Service.pdf
Downloading: https://www.utcourts.gov/howto/family/adoption/stepchild/docs/09_Finding_Conclusions.pdf
Downloading: https://www.utcourts.gov/howto/family/adoption/stepchild/docs/16_Order_Alternative_Service.pdf
skip
Downloading: https://www.utcourts.gov/howto/family/adoption/stepchild/docs/04_Consent_Guardian.pdf
skip
Downloading: https://www.utcourts.gov/howto/family/adoption/stepchild/docs/03_Consent_Non_Custodial_Parent.pdf
skip
Downloading: https://www.utcourts.gov/howto/family/adoption/stepchild/docs/17_Notice_Alternative_Service.pdf
skip
Downloading: https://www.utcourts.gov/howto/

Downloading: https://www.utcourts.gov/howto/informalprobate/docs/1008ES_Acceptance_of_Appointment.pdf
skip
Downloading: https://www.utcourts.gov/resources/forms/probate/affidavit_personal_property.pdf
skip
Downloading: https://www.utcourts.gov/howto/informalprobate/docs/1011ES_Announcement_of_Appointment_and_Notice_to_Creditors.pdf
skip
Downloading: https://www.utcourts.gov/howto/informalprobate/docs/1002ES_Application_Informal_Probate_With_Will.pdf
skip
Downloading: https://www.utcourts.gov/howto/informalprobate/docs/1001ES_Application_Informal_Probate_No_Will.pdf
skip
Downloading: https://www.utcourts.gov/howto/informalprobate/docs/Certificate_of_Service_Probate.pdf
skip
Downloading: https://www.utcourts.gov/howto/informalprobate/docs/1009ES_Letters_of_Administration.pdf
skip
Downloading: https://www.utcourts.gov/howto/informalprobate/docs/1010ES_Letters_Testamentary.pdf
skip
Downloading: https://www.utcourts.gov/howto/informalprobate/docs/1005ES_Objection_to_Appointment.pdf
skip
Dow

Downloading: https://www.utcourts.gov/media/docs/GRAMA_Request.pdf
skip
Downloading: https://www.utcourts.gov/howto/family/power_of_attorney_general/docs/02_Agent's_Certification.pdf
skip
Downloading: https://www.utcourts.gov/howto/family/power_of_attorney_general/docs/01_Power_of_Attorney.pdf
skip
Downloading: https://www.utcourts.gov/howto/family/power_of_attorney_general/docs/01_Power_of_Attorney_Spanish.pdf
skip
Downloading: https://www.utcourts.gov/howto/family/gc/guardianship/docs/10_Acceptance.pdf
skip
Downloading: https://www.utcourts.gov/howto/family/gc/docs/1251GE_Certification_of_Readiness_for_Trial_Probate.pdf
skip
Downloading: https://www.utcourts.gov/howto/family/gc/conservatorship/docs/08_Findings_Conclusions.pdf
skip
Downloading: https://www.utcourts.gov/howto/family/gc/conservatorship/docs/17_Letter.pdf
skip
Downloading: https://www.utcourts.gov/howto/family/gc/conservatorship/docs/04_Notice_of_Hearing.pdf
skip
Downloading: https://www.utcourts.gov/howto/family/gc/guar

In [21]:
display(files_df)

,id,jurisdiction,source,title,group,url,filename,downloaded
0,ed1efd08e3304bdaa887db753509d666,UT,www.utcourts.gov,Community Service Worksheet Third District Juv...,3rd District Juvenile Court: Forms and Pamphlets,https://www.utcourts.gov/courts/juv/juvsites/3...,COMMUNITY%20SERVICE%20WORKSHEET-FRONT%20AND%20...,2021-11-03
1,28ab1fb8df5c477cb4353e44f6d5a404,UT,www.utcourts.gov,Third District Juvenile Court Work Program Ref...,3rd District Juvenile Court: Forms and Pamphlets,https://www.utcourts.gov/courts/juv/juvsites/3...,Work_Crew_Application-2007.pdf,2021-11-03
2,279083cdac044d22b873a6a6ab8d9a14,UT,www.utcourts.gov,Utah State District Juvenile Court Probation O...,3rd District Juvenile Court: Forms and Pamphlets,https://www.utcourts.gov/courts/juv/juvsites/3...,Probation%20Order%20revised.050502.pdf,2021-11-03
3,ebad2caa6be0458d9e5c75635a7f66c2,UT,www.utcourts.gov,Adoptee's Consent to Adoption and Waiver of Ri...,Adopting a Minor Stepchild,https://www.utcourts.gov/howto/family/adoption...,02_Consent_Adoptee.pdf,2021-11-03
4,92932e4f938440ce8b93b6434aedfc3d,UT,www.utcourts.gov,Adoption Agreement,Adopting a Minor Stepchild,https://www.utcourts.gov/howto/family/adoption...,08_Agreement.pdf,2021-11-03
5,3bd98d56bbbe4d58b317a02d0f563f15,UT,www.utcourts.gov,Adoption Decree,Adopting a Minor Stepchild,https://www.utcourts.gov/howto/family/adoption...,10_Decree.pdf,2021-11-03
6,7a95ef55326f4ad199c485acc98cdb5f,UT,www.utcourts.gov,Birth Mother's Consent to Adoption and Waiver ...,Adopting a Minor Stepchild,https://www.utcourts.gov/howto/family/adoption...,03_1_Consent_Birth_Mother.pdf,2021-11-03
7,aa81b9bd14e446dbad87ccac1576f76a,UT,www.utcourts.gov,Certificate of Readiness for Adoption Hearing,Adopting a Minor Stepchild,https://www.utcourts.gov/howto/family/adoption...,07_Certificate_Readiness.pdf,2021-11-03
8,61a044e778024448b48447764752c374,UT,www.utcourts.gov,Ex Parte Motion for Alternative Service,Adopting a Minor Stepchild,https://www.utcourts.gov/howto/family/adoption...,13_Motion_Alternative_Service.pdf,2021-11-03
9,782b0e4f0ce240ffb11973034cb87ec0,UT,www.utcourts.gov,Findings of Fact and Conclusions of Law,Adopting a Minor Stepchild,https://www.utcourts.gov/howto/family/adoption...,09_Finding_Conclusions.pdf,2021-11-03


In [22]:
files_df.to_csv(csv_path+"form_data.csv", index=False, encoding="utf-8")

# Download forms and meta data from Michigan courts

Download forms and place them in the folder defined by `form_path` while creating a dataframe that you save to `csv_path`.


In [23]:
#from requests_html import HTMLSession
#from requests_html import AsyncHTMLSession
#import nest_asyncio
#import pyppdf.patch_pyppeteer

#asession = AsyncHTMLSession()
#r = await asession.get("https://www.courts.michigan.gov/SCAO-forms/appeals/")
#await r.html.arender(timeout=15, sleep=5)

In [24]:
jur = "MI"
source = "www.courts.michigan.gov"
form_path = "../data/raw/www.courts.michigan.gov/forms/"
csv_path = "../data/raw/www.courts.michigan.gov/"

today = date.today().strftime("%Y-%m-%d")

try:
    files_df = pd.read_csv(csv_path+"form_data.csv")
except:
    files_df = pd.DataFrame([],columns=["id","jurisdiction","source","title","group","url","filename","downloaded"])
    files_df.to_csv(csv_path+"form_data.csv", index=False, encoding="utf-8")
    
files_df[:1]

,id,jurisdiction,source,title,group,url,filename,downloaded
0,346d754e93274e83bfa8e73e0c651b1d,MI,www.courts.michigan.gov,Answer to Complaint to Recover Possession of P...,LTLC forms,https://www.courts.michigan.gov/4a205c/siteass...,dc111cnoinstructions.pdf,2021-11-03


The MI court site is all js rendered. So, I downloaded the rendered pages into `../data/raw/www.courts.michigan.gov/SCAO-forms/html/` and will deal with them there. I used `copy(document.body.innerHTML);` in console to grab the rendered page.

In [25]:
def MI_get(file):
    global files_df

    group = reCase(re.search(".*/([^/]*)\.html",file, re.IGNORECASE).group(1))

    with open(file) as f:
        contents = f.read()
    f.close()
    soup = BeautifulSoup(contents)
    body = soup.find('div',id="main-content")

    i = 0
    limit = 1
    for a in body.find_all('a', href=True):
        if (re.search(".*\.pdf",a['href'])):
            title = a.text.strip()
            url = re.search(".*/([^/]*)\.pdf",a['href'], re.IGNORECASE)
            filename = url.group(1)+".pdf"
            fileurl = "https://www.courts.michigan.gov"+url.group(0)
            fileid = uuid.uuid4().hex

            #print("\t",title,group,filename,"\n\t",fileurl,"\n")

            status = ""
            if fileurl not in files_df["url"].values:
                print("Downloading: "+fileurl)
                if (i < limit):
                    status = download_pdf(fileurl,form_path,fileid+".pdf")

                if status == "error":
                    print("#######################")
                    print("         Error!!!")
                    print("#######################")
                    time.sleep(5)
                else:
                    if (i < limit):
                        files_df = files_df.append(pd.DataFrame([[fileid,jur,source,title,group,fileurl,filename,today]],columns=["id","jurisdiction","source","title","group","url","filename","downloaded"]), ignore_index=True,sort=False)
                        i += 1
                    else:
                        print("skip")
                    #time.sleep(1)

In [26]:
path = "../data/raw/www.courts.michigan.gov/html/"
f = []
for (dirpath, dirnames, filenames) in walk(path):
    f.extend(filenames)
    break
f1 = [file for file in f]

for file in f1:
    if re.search(".*\.html",file):
        #print(path+file)
        MI_get(path+file)

Downloading: https://www.courts.michigan.gov/4a7a08/siteassets/forms/scao-approved/dc111c.pdf
Downloading: https://www.courts.michigan.gov/4a2228/siteassets/forms/scao-approved/dc111bnoinstructions.pdf
skip
Downloading: https://www.courts.michigan.gov/4a74bc/siteassets/forms/scao-approved/dc111b.pdf
skip
Downloading: https://www.courts.michigan.gov/4a74ca/siteassets/forms/scao-approved/dc111a.pdf
skip
Downloading: https://www.courts.michigan.gov/4a2499/siteassets/forms/scao-approved/dc111anoinstructions.pdf
skip
Downloading: https://www.courts.michigan.gov/4a2499/siteassets/forms/scao-approved/dc111dnoinstructions.pdf
skip
Downloading: https://www.courts.michigan.gov/4a79f3/siteassets/forms/scao-approved/dc111d.pdf
skip
Downloading: https://www.courts.michigan.gov/4a20f0/siteassets/forms/scao-approved/dc107.pdf
skip
Downloading: https://www.courts.michigan.gov/4a20fd/siteassets/forms/scao-approved/dc103.pdf
skip
Downloading: https://www.courts.michigan.gov/4a74dc/siteassets/forms/scao-

Downloading: https://www.courts.michigan.gov/4a7428/siteassets/forms/scao-approved/mc216a.pdf
skip
Downloading: https://www.courts.michigan.gov/4a2c41/siteassets/forms/scao-approved/mc06a.pdf
skip
Downloading: https://www.courts.michigan.gov/4a7c7c/siteassets/forms/scao-approved/mc211.pdf
skip
Downloading: https://www.courts.michigan.gov/4aa73d/siteassets/forms/scao-approved/mc267.pdf
skip
Downloading: https://www.courts.michigan.gov/4aa72b/siteassets/forms/scao-approved/mc254.pdf
skip
Downloading: https://www.courts.michigan.gov/4aa814/siteassets/forms/scao-approved/cc268.pdf
skip
Downloading: https://www.courts.michigan.gov/4aa7f0/siteassets/forms/scao-approved/cc269.pdf
skip
Downloading: https://www.courts.michigan.gov/4a2c69/siteassets/forms/scao-approved/dc91.pdf
skip
Downloading: https://www.courts.michigan.gov/4a2c5a/siteassets/forms/scao-approved/dc91a.pdf
skip
Downloading: https://www.courts.michigan.gov/4a7f33/siteassets/forms/scao-approved/cc297.pdf
skip
Downloading: https:/

Downloading: https://www.courts.michigan.gov/4a2bec/siteassets/forms/scao-approved/dc41.pdf
skip
Downloading: https://www.courts.michigan.gov/4a7de5/siteassets/forms/scao-approved/mc63.pdf
skip
Downloading: https://www.courts.michigan.gov/4a7ddb/siteassets/forms/scao-approved/mc65.pdf
skip
Downloading: https://www.courts.michigan.gov/4a2bdb/siteassets/forms/scao-approved/dc45.pdf
skip
Downloading: https://www.courts.michigan.gov/4aa77c/siteassets/forms/scao-approved/mc64.pdf
skip
Downloading: https://www.courts.michigan.gov/4a2be6/siteassets/forms/scao-approved/dc43.pdf
skip
Downloading: https://www.courts.michigan.gov/4a2c02/siteassets/forms/scao-approved/dc40.pdf
skip
Downloading: https://www.courts.michigan.gov/4a2bc6/siteassets/forms/scao-approved/dc46.pdf
skip
Downloading: https://www.courts.michigan.gov/4a2bcd/siteassets/forms/scao-approved/dc44.pdf
skip
Downloading: https://www.courts.michigan.gov/4a2bb1/siteassets/forms/scao-approved/dc47.pdf
skip
Downloading: https://www.court

Downloading: https://www.courts.michigan.gov/4a2cba/siteassets/forms/scao-approved/cc115.pdf
skip
Downloading: https://www.courts.michigan.gov/4a3e20/siteassets/forms/scao-approved/mc207.pdf
Downloading: https://www.courts.michigan.gov/4a6351/siteassets/forms/scao-approved/cc219b.pdf
skip
Downloading: https://www.courts.michigan.gov/4a81bf/siteassets/forms/scao-approved/mc219.pdf
skip
Downloading: https://www.courts.michigan.gov/4a3e24/siteassets/forms/scao-approved/mc263.pdf
skip
Downloading: https://www.courts.michigan.gov/4a7ecb/siteassets/forms/scao-approved/cc236.pdf
skip
Downloading: https://www.courts.michigan.gov/4a8180/siteassets/forms/scao-approved/mc294.pdf
skip
Downloading: https://www.courts.michigan.gov/4a3e75/siteassets/forms/scao-approved/mc262.pdf
skip
Downloading: https://www.courts.michigan.gov/4a30ef/siteassets/forms/scao-approved/cc378.pdf
Downloading: https://www.courts.michigan.gov/4a7d11/siteassets/forms/scao-approved/cc382.pdf
skip
Downloading: https://www.cour

Downloading: https://www.courts.michigan.gov/4aa60a/siteassets/forms/scao-approved/jc93.pdf
skip
Downloading: https://www.courts.michigan.gov/4a2bcb/siteassets/forms/scao-approved/jc99n.pdf
skip
Downloading: https://www.courts.michigan.gov/4a7d94/siteassets/forms/scao-approved/jc98n.pdf
skip
Downloading: https://www.courts.michigan.gov/4a2ba2/siteassets/forms/scao-approved/jc103.pdf
skip
Downloading: https://www.courts.michigan.gov/4aa61f/siteassets/forms/scao-approved/jc91.pdf
skip
Downloading: https://www.courts.michigan.gov/4aa5f5/siteassets/forms/scao-approved/jc95.pdf
skip
Downloading: https://www.courts.michigan.gov/4a2bb5/siteassets/forms/scao-approved/jc104.pdf
skip
Downloading: https://www.courts.michigan.gov/4aa5f8/siteassets/forms/scao-approved/jc100.pdf
skip
Downloading: https://www.courts.michigan.gov/4aa5f4/siteassets/forms/scao-approved/jc97.pdf
skip
Downloading: https://www.courts.michigan.gov/4a2bb8/siteassets/forms/scao-approved/jc99o.pdf
skip
Downloading: https://www

Downloading: https://www.courts.michigan.gov/4a7c32/siteassets/forms/scao-approved/foc63.pdf
skip
Downloading: https://www.courts.michigan.gov/4a7282/siteassets/forms/scao-approved/foc125.pdf
skip
Downloading: https://www.courts.michigan.gov/4a7e56/siteassets/forms/scao-approved/foc14.pdf
skip
Downloading: https://www.courts.michigan.gov/4a81b1/siteassets/forms/scao-approved/foc4.pdf
skip
Downloading: https://www.courts.michigan.gov/4a7297/siteassets/forms/scao-approved/foc123.pdf
skip
Downloading: https://www.courts.michigan.gov/4a7226/siteassets/forms/scao-approved/foc108.pdf
skip
Downloading: https://www.courts.michigan.gov/4a65f2/siteassets/forms/scao-approved/p05.pdf
skip
Downloading: https://www.courts.michigan.gov/4a7ee7/siteassets/forms/scao-approved/foc39e.pdf
skip
Downloading: https://www.courts.michigan.gov/4a7e45/siteassets/forms/scao-approved/foc13a.pdf
skip
Downloading: https://www.courts.michigan.gov/4a7f31/siteassets/forms/scao-approved/cc435.pdf
skip
Downloading: https

Downloading: https://www.courts.michigan.gov/4a7c38/siteassets/forms/scao-approved/pc606.pdf
skip
Downloading: https://www.courts.michigan.gov/4a8033/siteassets/forms/scao-approved/pc683m.pdf
skip
Downloading: https://www.courts.michigan.gov/4a8057/siteassets/forms/scao-approved/pc685m.pdf
skip
Downloading: https://www.courts.michigan.gov/4a8043/siteassets/forms/scao-approved/pc684m.pdf
skip
Downloading: https://www.courts.michigan.gov/4a7d84/siteassets/forms/scao-approved/pc570.pdf
skip
Downloading: https://www.courts.michigan.gov/4a802f/siteassets/forms/scao-approved/pc674.pdf
skip
Downloading: https://www.courts.michigan.gov/4a7fe5/siteassets/forms/scao-approved/pc645.pdf
skip
Downloading: https://www.courts.michigan.gov/4a7b76/siteassets/forms/scao-approved/pc599.pdf
skip
Downloading: https://www.courts.michigan.gov/4a7bd8/siteassets/forms/scao-approved/pc601.pdf
skip
Downloading: https://www.courts.michigan.gov/4a8003/siteassets/forms/scao-approved/pc648m.pdf
skip
Downloading: htt

Downloading: https://www.courts.michigan.gov/4a7f80/siteassets/forms/scao-approved/pc100.pdf
skip
Downloading: https://www.courts.michigan.gov/4a2bcf/siteassets/forms/scao-approved/pc102.pdf
skip
Downloading: https://www.courts.michigan.gov/4a2bc9/siteassets/forms/scao-approved/pc79.pdf
skip
Downloading: https://www.courts.michigan.gov/4a2cee/siteassets/forms/scao-approved/mc287.pdf
Downloading: https://www.courts.michigan.gov/4a7dd2/siteassets/forms/scao-approved/jc34.pdf
skip
Downloading: https://www.courts.michigan.gov/4a7d24/siteassets/forms/scao-approved/jc52.pdf
skip
Downloading: https://www.courts.michigan.gov/4a7d37/siteassets/forms/scao-approved/jc54.pdf
skip
Downloading: https://www.courts.michigan.gov/4a2f70/siteassets/forms/scao-approved/mc230.pdf
skip
Downloading: https://www.courts.michigan.gov/4aa456/siteassets/forms/scao-approved/jc60.pdf
skip
Downloading: https://www.courts.michigan.gov/4aa71c/siteassets/forms/scao-approved/mc391.pdf
skip
Downloading: https://www.court

Downloading: https://www.courts.michigan.gov/4a2ec9/siteassets/forms/scao-approved/mc227.pdf
skip
Downloading: https://www.courts.michigan.gov/4a2eb1/siteassets/forms/scao-approved/mc227a.pdf
skip
Downloading: https://www.courts.michigan.gov/4aa705/siteassets/forms/scao-approved/mc393.pdf
skip
Downloading: https://www.courts.michigan.gov/4a7ca3/siteassets/forms/scao-approved/mc240b.pdf
skip
Downloading: https://www.courts.michigan.gov/4a2ee7/siteassets/forms/scao-approved/mc208.pdf
skip
Downloading: https://www.courts.michigan.gov/4a7d66/siteassets/forms/scao-approved/mc292.pdf
skip
Downloading: https://www.courts.michigan.gov/4a7dcf/siteassets/forms/scao-approved/mc200a.pdf
skip
Downloading: https://www.courts.michigan.gov/4a7db7/siteassets/forms/scao-approved/mc200s.pdf
skip
Downloading: https://www.courts.michigan.gov/4a7dc7/siteassets/forms/scao-approved/mc200w.pdf
skip
Downloading: https://www.courts.michigan.gov/4a2fbb/siteassets/forms/scao-approved/mc205.pdf
skip
Downloading: ht

In [27]:
display(files_df)

,id,jurisdiction,source,title,group,url,filename,downloaded
0,346d754e93274e83bfa8e73e0c651b1d,MI,www.courts.michigan.gov,Answer to Complaint to Recover Possession of P...,LTLC forms,https://www.courts.michigan.gov/4a205c/siteass...,dc111cnoinstructions.pdf,2021-11-03
1,57f56df055254cde9ecd9ce217b9dca3,MI,www.courts.michigan.gov,Addendum A to Verified Declaration About Pater...,adoptions court forms,https://www.courts.michigan.gov/49c512/siteass...,pca356a.pdf,2021-11-03
2,5e89ca0faf59435d9d668abe9704b37f,MI,www.courts.michigan.gov,"14-Day Notice, Traffic",motor vehicle offenses forms,https://www.courts.michigan.gov/4a70d4/siteass...,mc216.pdf,2021-11-03
3,92df7834768f4510968315422054e74e,MI,www.courts.michigan.gov,Motion and Order for Discharge From Probation,probation,https://www.courts.michigan.gov/4a2d6f/siteass...,mc245m_245o.pdf,2021-11-03
4,79bda242486243a5ba8e6496b4e80305,MI,www.courts.michigan.gov,"Bond, Claim and Delivery",claim and delivery,https://www.courts.michigan.gov/4a2da7/siteass...,mc38.pdf,2021-11-03
...,...,...,...,...,...,...,...,...
65,2ae35d7dfa7e465c99736a9740fd4560,MI,www.courts.michigan.gov,Judgment of Sentence/Commitment to Department ...,juvenile designated forms,https://www.courts.michigan.gov/4a7df7/siteass...,jc72.pdf,2021-11-03
66,fd9424f053a44e8280befd0d365c6ca1,MI,www.courts.michigan.gov,Order after Hearing on Petition to Accept Rele...,safe delivery newborn,https://www.courts.michigan.gov/4a2dc0/siteass...,ccfd08.pdf,2021-11-03
67,fa8c956052f94a78a0c951328006bd62,MI,www.courts.michigan.gov,Case Evaluator Application,case evaluation,https://www.courts.michigan.gov/4a2af9/siteass...,mc34.pdf,2021-11-03
68,1ceaeb87496540e1b0b46929498536fd,MI,www.courts.michigan.gov,Collecting Your Money from a Small Claims Judg...,small claims,https://www.courts.michigan.gov/4a801f/siteass...,dci84.pdf,2021-11-03


In [28]:
files_df.to_csv(csv_path+"form_data.csv", index=False, encoding="utf-8")